Kyle Shannon 1/1/16 - Project 3 - Data Wrangling with Mongodb

#### Disclaimer about PDF

I tried to save this Ipython file as a PDF, but I get an error. I tried to solve this by installing pandoc via homebrew, as suggested here: http://stackoverflow.com/questions/29156653/ipython-jupyter-problems-saving-notebook-as-pdf

However, I am still encountering an error, and do not have the time to solve it at the moment. I hope this html document is alright with Udacity. I have used an html doc from ipython to submit all my other projects so far. Thankyou!

### Overview

I choose to download city data of Seattle, Washington from [https://mapzen.com/data/metro-extracts/]; however the data was too large to directly export from here: [https://www.openstreetmap.org/relation/237385]. While I live in San Diego and have never lived in Seattle, I choose this dataset because my girlfriend and have begun planning a trip to Seattle. I figured this project would be helpful in learning at a high level a bit more about Seattle and what the city has to offer.

#### Description of Files in Folder

- **project3_dataWrangling.py** This is the main python file that creates the mongodb, loads the seattle_washington.osm file, parses it, and inserts it into the database. 
- **db_query.py** queries the database. There is one pipeline in the file; however, below the code in comments are all the pipelines I wrote with sample outputs. This way, I can simply copy paste a pipeline if I wish to run an older query.
- **xml_decimate.py** The meat of this code was provided by the course instructor as a way of decimating a .osm file. This was useful for testing purposes as it is much faster combing through a tenth of the data.
- **Helper scripts .py files:** There are several scripts in here that I used along with Grep in the terminal to gather facts about the .osm file. I primarily ran these .py files and grepped through the decimated .osm file as it took about a .10th of the time. This helped me learn more about the data I was about to parse and what type of exceptions I might run into and encounter. This helped me write the main parsing function(s). 
- **tiny_seattle.osm** This dataset is a small sample set that the rubric asked me to include. I just ran the sample_seattle.osm file through the xml_split.py file.
- **sample_seattle.osm** This osm file is the decimated dataset from seattle_washington.osm. I used this dataset to run my python scripts and grep commands against. It provided me a good approximation of what I might run into when parsing the large .osm file. (this file is not included in the folder due to size)
- **seattle_washington.osm** The large ~1.5gb .osm file for Seattle. (this file is not included in the folder due to size)

#### Approach Taken During Project

Decimating the large .osm file was first. Leaving me with an xml file ~150.0 mb. Using code from lesson 6 exercises I wrote several scripts to find info about the data. E.g. unique zipcodes, country codes and road types. Combining this with Grep e.g.: 
    
    Kyles-MBP:project 3 kyleshannon$ grep -e -A 10 -B 10 mexican sample_seattle.osm | less 

which returns a line of xml data containing the string 'mexican' along with the 10 lines before it and after it. This combination provided me a time effective way of learning about the data set before writing my main parsing script. The python file that handles parsing the data also handles creating and inserting documents into a mongodb. E.g.: 
    
    def process_osm(file_in):
        with open(file_in) as file: #opens and closes the xml file.
            for _, element in ET.iterparse(file): #iterating through .osm file
                el = shape_data(element) #parsing data elements one by one
                if el:
                    #pprint.pprint(el)
                    way_node_collection.insert(el) #writing parsed data into mongodb

This script took ~20 min to run on a ~1.5gb xml file. After that my mongodb was created and the documents were already inserted into it. I could then begin to query the database.

### Problems Encountered in your Map. 

During the pre-query phase of the project there were several issues that I decided the let the parser handle. This included wrong country codes, city names, zip codes, standardizing road type names, weird address data and what I call Source scope creep. These are discussed in detail below.

#### Country Codes/City Names/Postcodes

Because Seattle is so close to Canada, I wanted to make sure no Canadian data snuck through. Running:

    grep -e country_code sample_seattle.osm | less

returned a list of country codes in the dataset, I noticed several Canadian codes. Therefore I wrote a placeholder function to check 

    country code.lower() == 'us'. 
    
The same process occurred for city names. I noticed several city names (Tacoma, Kirkland and so on.) These may be in the county of Seattle (King's County), but they are not part of the city of Seattle. Therefore I decided not to include them.

A final check included postcodes (zipcodes). I went online and found the postcodes associated with the city of Seattle. I used a python script to check if any postcodes not in this list were in the sample_seattle.osm dataset:

    import xml.etree.cElementTree as ET
    import pprint

    #zipcodes sourced from http://zipcode.org/city/WA/SEATTLE
    wa_zipcodes = ['98101', ... '98198']

    def get_zipcode(element):
        return element.get('postcode')

    def unique_users(filename):
        erras = set()
        count = 0
        for _, element in ET.iterparse(filename):
            if count = 10:
                break
            if get_zipcode(element) in wa_zipcodes:
                continue
            erras.add(get_zipcode(element))
            count += 1
        print erras

    xml_file = "sample_seattle.osm"
    unique_users(xml_file)

Sure enough there were other postcodes. This became another function in my main parser. How I handled catching data I did not want to insert, was by setting a flag at the top of my main data parsing function: is_valid = True. If any of these cases were found, then the flag would become false and at the end of the function the parsed data would not be returned, thus not be inserted into mongo.

#### Unifying Road Types

Again, I used grep along with a simple script to collect info about the various road types. Then I created a dict called MAPPING which contained key/value pairs. E.g. 

    {'st':'Street', 'st.':'Street', ...}

#### Weird Addresses

Addresses provided a bit of a headache due to its insidious nature. For example some addresses were easy to flag as they followed an: 'addr:xyz'; however some had multiple colons ':' in them. When addr's contained multiple colons they appeared to be duplicates, so I decided to ignore them and not include them in the database and continue along with the for loop:

    if key.startswith('addr:'):
		if ':' in key[5:]:
            continue

Some entries had a flag in them called 'fixme'. I decided to throw out all of these entries completely. If there was one issue with parts of the data, then how could I trust the rest of the document being written. Code below:

	if key.lower() == 'fixme':
		is_valid = False
		continue
It is during this part of the parsing that I check if the address is in Seattle by passing the element's key into the country, city and postcode functions: 

	if addr_key.lower() == 'country':
		is_valid = is_valid_country(val)
					
    if addr_key.lower() == 'city':
		is_valid = is_valid_city(val)
					
	if addr_key.lower() == 'postcode':
		is_valid = is_valid_postcode(val)
				
	if not is_valid:
		continue
        
Another check I perform is that an address was applied to the empty address dictionary. If not then I do not want to create an empty address, that would create an empty address document in the database, which we do not want:

    if len(address) > 0:
        node['address'] = address

Once these addresses check out then I wrote them into the address dictionary and continue on with the for loop through the element.

#### Source Scope Creep

One interesting I found was what I am calling source scope creep. What occurs is when there was a source 'tiger_import' there was accompanying 'tiger:xyz' duplicates for preexisting 'addr:xyz' fields. I was thinking about removing all of these duplicates, but in the end I decided to keep the tiger data as it did not interfere with my data and it might be useful as the tiger data did include other fields that were unique. But I call it scope creep because the source which is Tiger is finding its way into fields like postcode and county. Where it should not be. 

    
    <tag k="tiger:source" v="tiger_import_dch_v0.6_20070830" />
    <tag k="tiger:county" v="Clallam, WA" />
    <tag k="addr:county" v="Clallam, WA" />

### Overview of the Data

- Size of the complete Seattle.osm file: 1.52gb
- Number of nodes: 6762600 and number of ways: 614082

        print ("number of nodes: %d and number of ways: %d" 
            %(db.count({'type' : 'node'}),
			db.count({'type' : 'way'})))
            
- Top 5 active users:

        pipeline = [ 
                    {'$group' : {'_id' : '$created.user',
			 	                 'count' : {'$sum' : 1 }}},
                    {'$sort' : {'count' : -1}},
                    {'$limit' : 25}]
		returns:
				{u'_id': u'Glassman', u'count': 1217504}
                {u'_id': u'SeattleImport', u'count': 749345}
				{u'_id': u'tylerritchie', u'count': 667752}
				{u'_id': u'woodpeck_fixbot', u'count': 634558}
				{u'_id': u'alester', u'count': 311665} 

- Top 5 appearing amenities:
		
        pipeline = [
                    {'$match' : {'amenity' : {'$exists' : 1}}},
                    {'$group' : {'_id' : '$amenity',
			                     'count' : {'$sum' : 1 }}},
			        {'$sort' : {'count' : -1}},
			        {'$limit' : 20}]
        Returns:
				{u'_id': u'parking', u'count': 7860}
				{u'_id': u'bicycle_parking', u'count': 3059}
				{u'_id': u'school', u'count': 2520}
				{u'_id': u'restaurant', u'count': 2257}
				{u'_id': u'bench', u'count': 1713}

- Unique source counts:

		pipeline = [ 
					{'$group' : {'_id' : '$source',
		                         'count' : {'$sum' : 1 }}},
		            {'$sort' : {'count' : -1}},
		            {'$limit' : 4}]
        returns:
					{u'_id': None, u'count': 6917511}
					{u'_id': u'King County GIS;data.seattle.gov', u'count': 183167}
					{u'_id': u'data.seattle.gov', u'count': 56847}
					{u'_id': u'Bing', u'count': 39542}

I was really interested in the bike amenities data. So I used Grep and mongodb to find out some high level info about bikes:

    grep -c bicycle_parking seattle_washington.osm
        Returns: 3210
    grep -e bicycle seattle_washington.osm
        Sample return:
        <tag k="bicycle" v="yes"/> or v="no", v="designated", v="private", v="limited", v="permissive"/>
        <tag k="shop" v="bicycle"/>
        <tag k="amenity" v="bicycle_parking"/>
        <tag k="route" v="bicycle"/>
        
- Ordered list of unique users contributing bike info:

        pipeline = [
                {"$match": {"amenity": "bicycle_parking"}},
                {'$group' : {'_id' : '$created.user',
				             'count' : {'$sum' : 1 }}},
                {'$sort' : {'count' : -1}}]
			
        returns:
				{u'_id': u'WBSKI', u'count': 2279}
				{u'_id': u'zephyr', u'count': 166}
				{u'_id': u'seattlefyi', u'count': 90}
                ...


Lastly, I looked into food, because who doesn't like discovering more about food!

- Checking out unique cusines by count:

        pipeline = [ 
					{'$match' : {'amenity' : 'restaurant'}},
					{'$group' : {'_id' : '$cuisine',
			                     'count' : {'$sum' : 1 }}},
			        {'$sort' : {'count' : -1}},
			        {'$limit' : 20}]
        Returns:

					{u'_id': None, u'count': 825}
					{u'_id': u'mexican', u'count': 163}
					{u'_id': u'pizza', u'count': 160}
					{u'_id': u'american', u'count': 120}
					{u'_id': u'chinese', u'count': 105}
					{u'_id': u'thai', u'count': 100}
					{u'_id': u'asian', u'count': 86}
					{u'_id': u'italian', u'count': 83}
					{u'_id': u'japanese', u'count': 77}
					{u'_id': u'sandwich', u'count': 56}
					{u'_id': u'vietnamese', u'count': 53}
					{u'_id': u'seafood', u'count': 43}


### Other Ideas About the Datasets

Looking at the top 5 active users we can see that perhaps a few bots are automating data, e.g. woodpeck_fixbot as well as SeattleImport. However, there are users like Glassman (top user) and Tylerritchie that are very active. Perhaps they spent sometime building an automated process to add all of their data to openstreetmaps. 

There are many exciting questions you can ask and ways to make use of such rich data. I would recommend taking a closer look at the  'bicycle_parking' amenity. The first interesting thing to note is that bike_parking is the 2nd most numerous amenity after car parking. I was not sure how bike friendly Seattle would be, especially due to the number of days it rains. Lets say we have a client that wants to set up mobile repair/snack bike shops and they obviously want to target cyclists that are in need of repair. They also want to be able to travel along frequented bike paths and know the landscape of competitive brick and mortar bike shops. Here is what I would suggest to such a client:

- Take a look at the bike data we currently have and web scrape to fill in potential holes. Look for additional bike parking locations, well-known bike paths and bike shops. Add missing ones to the database.
- Next we could use postcodes, or using long/lat, create a heatmap of bike paths, parking and shops over the city of Seattle. 
- Look for any trends in the data, perhaps there are a few large bike paths that have many bike parking areas around it and a few bike shops in close proximity.
- Suggest that sticking to these paths might produce the most positive customer interactions/transactions. 

Obviously the .osm data is not very granular and would be difficult to extend to such a specific client problem. However, I believe it does provide a good backbone from which we can add more specific cyclist related data. Then from a more complete data set, we could provide better predictions and cyclist insight. The strength of the .osm data set is in its generality. Once you have it, it can be used in many ways. Perhaps cyclists frequent cafes after their morning ride? Well guess what we have lots of data on the placement of cafes. What cafes are close to bike paths? Then we can go online, or even call the cafe shops and ask if it’s a hot spot for cyclists. The point is that we can use this general dataset to drill down into the bedrock of our questions and provide guidance on where to focus our efforts.


There were many entries that were tagged as 'Fixme'. Upon further investigation I found that these are notes-to-self placed by contributors. ref: http://wiki.openstreetmap.org/wiki/Key:fixme

Restaurants

I love coffee shops, so I took a look at the type of coffee shops in Seattle, grouped by 'cuisine': 

        {u'_id': None, u'count': 487}
        {u'_id': u'coffee_shop', u'count': 332}
        {u'_id': u'ice_cream', u'count': 17}
        {u'_id': u'sandwich', u'count': 15}
        {u'_id': u'american', u'count': 9}
        {u'_id': u'tea', u'count': 8}
        {u'_id': u'coffee', u'count': 6}
        {u'_id': u'vietnamese', u'count': 3}
        {u'_id': u'frozen_yogurt', u'count': 3}
        {u'_id': u'juice', u'count': 3}
        {u'_id': u'donut', u'count': 3}
        {u'_id': u'sushi', u'count': 3}
        {u'_id': u'chocolate', u'count': 3}
        {u'_id': u'breakfast', u'count': 2}
        {u'_id': u'greek', u'count': 2}
        {u'_id': u'pizza', u'count': 2}
        {u'_id': u'italian', u'count': 2}
        {u'_id': u'pies', u'count': 2}
        {u'_id': u'asian', u'count': 2}
        {u'_id': u'seafood', u'count': 2}
        
Many of these coffee_shops I can see being coffee shops, like cuisine 'donut', 'breakfast' and so on. But some are a mystery, like 'seafood' etc.  So I then grouped not by 'cuisine', but by 'name' and this was the list:

        {u'_id': u'Starbucks', u'count': 206}
        {u'_id': None, u'count': 44}
        {u'_id': u"Tully's", u'count': 9}
        {u'_id': u'Espresso', u'count': 8}
        {u'_id': u'Uptown Espresso', u'count': 8}
        {u'_id': u'Starbucks Coffee', u'count': 8}
        {u'_id': u"Tully's Coffee", u'count': 7}
        {u'_id': u"Starbuck's", u'count': 7}
        {u'_id': u'Serious Coffee', u'count': 6}
        {u'_id': u'Caffe Ladro', u'count': 6}
        {u'_id': u'Cherry Street Coffee House', u'count': 5}
        {u'_id': u'Tim Hortons', u'count': 5}
        {u'_id': u'Diva Espresso', u'count': 5}
        {u'_id': u'Top Pot Doughnuts', u'count': 4}
        {u'_id': u'Herkimer Coffee', u'count': 4}
        {u'_id': u'Caffe Vita', u'count': 4}
        {u'_id': u'Jamba Juice', u'count': 4}
        {u'_id': u"Specialty's", u'count': 4}
        {u'_id': u'Chocolati Caf\xe9', u'count': 3}
        {u'_id': u'Emerald City Smoothie', u'count': 3}
        
No surprise there... Starbucks in the front, along with a few other spellings. I could go through and fix these.

So what ratio of cafes are Starbucks?

	print ("number of starbucks: %d and number of cafes: %d" 
				%(db.count({'name' : 'Starbucks'}),
				db.count({'amenity' : 'cafe'})))
    
    Prints:
        number of starbucks: 213 and number of cafes: 947

Roughly 22%, I probably did not catch all of the Starbucks, due to a few entries, having different spelling. 

When I fgrep for the non case sensitive string literal 'starbuck' I get a count of: 319.

        Kyles-MBP:project 3 kyleshannon$ cat seattle_washington.osm | fgrep -i -c "Starbuck" | less

I would need to investigate the data more closely to find where these other occurrences of Starbuck comes from. But I am grepping the original OSM file, so this is just a loose approximation.



### Conclusion

Seattle was a fairly clean data set, especially street names. There was some standardizing that occurred for road types, and I programmatically removed data not in US, Seattle, and Seattle's zipcode list. I also removed data that were flagged as 'fixme' because that tag symbolized to me incomplete data. I did most of the data removal/changing in the parsing stage and used grep + db querying to discover insight about the data set. Obviously there are many questions and projects one could tackle with a cleaned open street map data set. I choose to focus effort of cafes, bike amenities and I also checked out restaurants. There were a surprising number of mexican restaurants. Upon further investigation I found that over 1/3 of them were taco bells!

### Sources/Resources Used

Documentation for:

- Grep
- Mongodb
- Python 2.x
- regex
- openstreetmaps

Sites & books: 

- http://regexr.com/  #one of the best sites ever...
- orielly's mongodb guide http://shop.oreilly.com/product/0636920028031.do
- http://enginerds.craftsy.com/blog/2014/04/parsing-large-xml-files-in-python-without-a-billion-gigs-of-ram.html
- http://www.ibm.com/developerworks/xml/library/x-hiperfparse/
- http://zipcode.org/city/WA/SEATTLE

Code: 

- a lot of code from the online class (code to decimate the big .osm file as well as lessomn 6 code and mongodb code from other lessons)

